In [1]:
# Imports & Config
import pandas
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
from logger import log

preferred_cycle_time = 15  # this value is chosen because this is the median, mean and mode value in the data.
permitted_difference = 120  # 120 seconds - controls max cycle gap time before we consider a race sequence useless.

In [2]:
def get_whole_file(file_path: str) -> pandas.DataFrame:
    df = pandas.read_csv(file_path, dtype={'asatdatestr': str,  # bet cycle date
                                           'asattimestr': str,  # bet cycle time
                                           # These ids probably form a unique composite of some sort
                                           'meetid': str,  # 2 unique ids
                                           'performanceid': str,  # 290 unique ids
                                           'raceid': str,  # 11 unique ids
                                           # Seems to be combo of asatdate+meetid+performanceid+raceid
                                           'refid': str,
                                           'type': str,  # FINAL or INTER, but what does it mean?
                                           # bet type: place, win, quinella and omni.
                                           # Different types allow different bet combinations and accordingly pool dimensions.
                                           'pool': str,
                                           'ccy': str,  # currency: 83% HKD and 17% GBP
                                           'fectaflag': str,  # no idea what this is, but always G
                                           'dimension': str,
                                           # dimensionality of bet data in below entries.
                                           'dimension_x': str,  # dimension x value
                                           'dimension_y': str,  # dimension y value
                                           # horse number  and bet amount
                                           # is the number of participating horses fixed or variable?
                                           'horse_nb_1': str, 'bet_amount_horse_nb_1': float,
                                           'horse_nb_2': str, 'bet_amount_horse_nb_2': float,
                                           'horse_nb_3': str, 'bet_amount_horse_nb_3': float,
                                           'horse_nb_4': str, 'bet_amount_horse_nb_4': float,
                                           'horse_nb_5': str, 'bet_amount_horse_nb_5': float,
                                           'horse_nb_6': str, 'bet_amount_horse_nb_6': float,
                                           'horse_nb_7': str, 'bet_amount_horse_nb_7': float,
                                           'horse_nb_8': str, 'bet_amount_horse_nb_8': float,
                                           'horse_nb_9': str, 'bet_amount_horse_nb_9': float,
                                           'horse_nb_10': str, 'bet_amount_horse_nb_10': float,
                                           'horse_nb_11': str, 'bet_amount_horse_nb_11': float,
                                           'horse_nb_12': str, 'bet_amount_horse_nb_12': float,
                                           'horse_nb_13': str, 'bet_amount_horse_nb_13': float,
                                           'horse_nb_14': str, 'bet_amount_horse_nb_14': float,
                                           'asatdate': str,  # unknown date
                                           'asatdatetime': str,  # unknown timestamp
                                           # irrelevant DB properties
                                           'uploadtimestamp': str,
                                           'filepath': str,
                                           'filename': str,
                                           })
    log(f"Read in file of {df['refid'].nunique()} races, comprising {len(df.index)} rows")
    return df

data = get_whole_file('../data/0_raw/query_result.csv')

17:30:16: Read in file of 3163 races, comprising 15648648 rows


In [3]:
def pick_win_data(df: pandas.DataFrame) -> pandas.DataFrame:
    win_only = df.loc[df['pool'] == 'WIN']
    relevant_columns = win_only[
        ['asattimestr', 'refid', 'type', 'ccy',
         'bet_amount_horse_nb_1', 'bet_amount_horse_nb_2', 'bet_amount_horse_nb_3', 'bet_amount_horse_nb_4',
         'bet_amount_horse_nb_5', 'bet_amount_horse_nb_6', 'bet_amount_horse_nb_7', 'bet_amount_horse_nb_8',
         'bet_amount_horse_nb_9', 'bet_amount_horse_nb_10', 'bet_amount_horse_nb_11', 'bet_amount_horse_nb_12',
         'bet_amount_horse_nb_13', 'bet_amount_horse_nb_14']]

    log(f'Picked winning pool of {len(relevant_columns)} rows')
    return relevant_columns

data = pick_win_data(data)

17:30:18: Picked winning pool of 682640 rows


In [4]:
def create_unique_composite_row_id(df: pandas.DataFrame) -> pandas.DataFrame:
    df['_id'] = df['refid'] + '-' + df['asattimestr']
    return df

data = create_unique_composite_row_id(data)

In [5]:
def fill_empty_bets(df: pandas.DataFrame) -> pandas.DataFrame:
    for i in range(1, 15):
        column = f'bet_amount_horse_nb_{i}'
        df.loc[df[column].isnull(), column] = 0
    log(f'Added default 0 bet pools for missing horses')

    return df

data = fill_empty_bets(data)

17:30:18: Added default 0 bet pools for missing horses


In [6]:
def convert_currencies(df: pandas.DataFrame) -> pandas.DataFrame:
    for i in range(1, 15):
        column = f'bet_amount_horse_nb_{i}'
        df.loc[df['ccy'] == 'GBP', column] = df.loc[df['ccy'] == 'GBP', column] * 10
    df.loc[df['ccy'] == 'GBP', 'ccy'] = 'HKD'

    log(f'Converted all bet currencies to HKD')
    return df

data = convert_currencies(data)


17:30:20: Converted all bet currencies to HKD


In [7]:
def sort_race_time_series(df: pandas.DataFrame) -> pandas.DataFrame:
    sorted_df = df.sort_values(['_id'], ascending=[True])
    log(f'Sorted data by date, id and time order')
    return sorted_df

data = sort_race_time_series(data)


17:30:20: Sorted data by date, id and time order


In [8]:
# Fixes strange cases where a bet pool spikes from 0 to a huge amount in one cycle and then goes back to 0.
# The fix here, is just to set them to 0 as this data is noise.
def fix_bet_spikes(df: pandas.DataFrame) -> pandas.DataFrame:
    fixed_entries = 0
    races = df['refid'].unique()
    for refid in tqdm(races):
        race = df.loc[df['refid'] == refid]
        first_row = second_row = None
        second_row_index = None

        # We're passing over the rows in triplets as a window to check for the spike.
        for index, third_row in race.iterrows():
            if first_row is not None and second_row is not None:
                for i in range(1, 15):
                    column = f'bet_amount_horse_nb_{i}'
                    if first_row[column] == 0 and second_row[column] != 0 and third_row[column] == 0:
                        df.loc[second_row_index, column] = 0
                        fixed_entries += 1

            first_row = second_row
            second_row = third_row
            second_row_index = index

    log(f'Corrected {fixed_entries} bet pool entries which spiked from 0 to some huge amount, then back to 0 for no '
        f'reason.')

    return df

data = fix_bet_spikes(data)


100%|██████████| 3163/3163 [03:16<00:00, 16.08it/s]

17:33:37: Corrected 14 bet pool entries which spiked from 0 to some huge amount, then back to 0 for no reason.


In [9]:
# Expects a sorted df. Removes rows that chronologically appear after FINAL cycle of a race.
def remove_invalid_timestamp_rows(df: pandas.DataFrame) -> pandas.DataFrame:
    final_cycles = df.loc[df['type'] == 'FINAL']
    invalid_indices = []
    for index, row in tqdm(final_cycles.iterrows()):
        invalid_rows = df.loc[(df['refid'] == row['refid']) & (df['asattimestr'] > row['asattimestr'])]
        for i, r in invalid_rows.iterrows():
            invalid_indices.append(i)

    df.drop(index=invalid_indices, inplace=True)
    log(f'Removed {len(invalid_indices)} rows with bet pool timestamps after the FINAL bet cycle')

    return df

data = remove_invalid_timestamp_rows(data)


3181it [05:06, 10.39it/s]


17:38:43: Removed 38 rows with bet pool timestamps after the FINAL bet cycle


In [10]:
# Removes rows whose cycles are too discontiguous (their cycles are to irregular)
def remove_too_discontiguous_cycles(df: pandas.DataFrame) -> pandas.DataFrame:
    removed_row_count = 0
    races = df['refid'].unique()
    indices_to_remove = []

    for refid in tqdm(races):
        race = df.loc[df['refid'] == refid]
        durations = pandas.to_timedelta(race['asattimestr']).dt.total_seconds()
        visited_durations = []
        visited_indices = []

        for index, duration in durations.iteritems():
            if visited_durations:
                if duration - visited_durations[-1] > permitted_difference:
                    indices_to_remove += visited_indices
                    visited_durations = []
                    visited_indices = []
            visited_durations.append(duration)
            visited_indices.append(index)

    df.drop(index=indices_to_remove, inplace=True)
    removed_row_count += len(indices_to_remove)

    log(f'Removed {removed_row_count} rows, part of sequences with at least one cycle gap > {permitted_difference} seconds.')

    return df

data = remove_too_discontiguous_cycles(data)



100%|██████████| 3163/3163 [02:10<00:00, 24.30it/s]


17:40:54: Removed 54187 rows, part of sequences with at least one cycle gap > 120 seconds.


In [11]:
# Remove races with too few datapoints, as we'd have to pad with a lot of synthetic data for them to be usable,
# at which point they will not add much value anyway.
def remove_races_with_insufficient_data(df: pandas.DataFrame) -> pandas.DataFrame:
    minimum_cycle_count = 25
    removed_rows = 0
    removed_races = 0
    races = df['refid'].unique()

    for refid in tqdm(races):
        race = df.loc[df['refid'] == refid]
        if len(race.index) < minimum_cycle_count:
            df = df[df['refid'] != refid]
            removed_races += 1
            removed_rows += len(race.index)

    log(f'Removed {removed_races} races ({removed_rows} rows) due to insufficient cycle data vs. '
        f'the minimum of {minimum_cycle_count}')

    return df

data = remove_races_with_insufficient_data(data)


100%|██████████| 3163/3163 [02:05<00:00, 25.28it/s]

17:42:59: Removed 57 races (57 rows) due to insufficient cycle data vs. the minimum of 25


In [12]:
# Returns the pair of rows that a given timestamp would best fit between.
def linear_search_timestamps(race: pandas.DataFrame, timestamp: int) -> [pandas.Series, pandas.Series]:
    later_timestamps = race.loc[race['_seconds'] >= timestamp]
    earlier_timestamps = race.loc[race['_seconds'] < timestamp]

    if len(later_timestamps.index) == 0 or len(earlier_timestamps.index) == 0:
        return None

    return earlier_timestamps.iloc[-1], later_timestamps.iloc[0]


def generate_new_row(lower_row: pandas.Series, upper_row: pandas.Series, lower_weight: float, upper_weight: float,
                     chosen_timestamp: float) -> dict:
    new_row = {}
    for column, value in lower_row.iteritems():
        if column == 'asattimestr':
            new_row[column] = str(timedelta(seconds=chosen_timestamp))
        elif column == 'refid':
            new_row[column] = lower_row[column]
        elif column == 'type':
            new_row[column] = lower_row[column]
        elif column == 'ccy':
            new_row[column] = lower_row[column]
        elif column == '_id':
            new_row[column] = lower_row['refid'] + '-' + str(timedelta(seconds=chosen_timestamp))
        elif 'bet_amount_horse_nb_' in column:
            new_row[column] = lower_weight * lower_row[column] + upper_weight * upper_row[column]

    return new_row

def synthesize_contiguous_timestamps(df: pandas.DataFrame) -> pandas.DataFrame:
    df['_seconds'] = pandas.to_timedelta(df['asattimestr']).dt.total_seconds()
    races = df['refid'].unique()

    all_synthesized_rows = []
    for refid in tqdm(races):
        race = df.loc[df['refid'] == refid]
        final_timestamp = race.iloc[-1]['_seconds']
        next_timestamp = final_timestamp
        synthesized_rows = []
        while True:
            next_timestamp -= preferred_cycle_time
            surrounding_row_pair = linear_search_timestamps(race, next_timestamp)
            # No more rows to synthesize
            if surrounding_row_pair is None:
                break

            [lower_row, upper_row] = surrounding_row_pair
            lower_bound_distance = next_timestamp - lower_row['_seconds']
            upper_bound_distance = upper_row['_seconds'] - next_timestamp
            # We subtract it from 1 as we want to increase the weight inversely to the distance.
            # (i.e lower distance => higher weight)
            normalised_lower = 1 - (lower_bound_distance / (upper_row['_seconds'] - lower_row['_seconds']))
            normalised_upper = 1 - (upper_bound_distance / (upper_row['_seconds'] - lower_row['_seconds']))
            new_row = generate_new_row(lower_row, upper_row, normalised_lower, normalised_upper, next_timestamp)
            synthesized_rows.append(new_row)

        final_row = race.iloc[-1]
        synthesized_rows.reverse()
        synthesized_rows.append(generate_new_row(final_row, final_row, 0, 1, final_row.loc['_seconds']))
        all_synthesized_rows += synthesized_rows

    result = pandas.DataFrame.from_records(all_synthesized_rows)

    log(f'Synthesised {len(all_synthesized_rows)} new rows for {races.size} races to space them into '
        f'{preferred_cycle_time}s gaps')

    return result

data = synthesize_contiguous_timestamps(data)

100%|██████████| 3106/3106 [13:18<00:00,  3.89it/s]


17:56:21: Synthesised 634679 new rows for 3106 races to space them into 15s gaps


In [13]:
# A basic check to ensure the rows are in order is to check that bet total amounts only ever increase for a race.
# This should work as money can only be added to betting pools, not removed.
def validate(df: pandas.DataFrame):
    current_race = None
    max_drop_per_cycle = 100000
    betting_array = [0] * 15
    invalid_count = 0
    deviations = {}
    for index, row in tqdm(df.iterrows()):
        out_of_order = False
        if current_race != row['refid']:
            current_race = row['refid']
            betting_array = [0] * 15
        for i in range(1, 15):
            column = f'bet_amount_horse_nb_{i}'
            # Considering only rows with discrepancies > 100k USD an issue worth looking into.
            if row[column] < betting_array[i - 1] - max_drop_per_cycle:
                out_of_order = True
                diff = row[column] - betting_array[i - 1]
                deviations[diff] = (deviations.get(diff) or 0) + 1
            betting_array[i - 1] = row[column]

        if out_of_order:
            df.loc[index, '_suspicious'] = True
            invalid_count += 1

    log(f'Found {invalid_count} rows with betting pools that drop by >{max_drop_per_cycle} HKD in one step')
    log('Validation complete.')

validate(data)


634679it [01:20, 7932.77it/s]

17:57:42: Found 57 rows with betting pools that drop by >100000 HKD in one step
17:57:42: Validation complete.


In [14]:
date_string = datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
output_path = f'../data/win_only_data-{date_string}.csv'

log(f'Writing processed file of {len(data.index)} rows to {output_path}')

data.to_csv(output_path)

log(f'Finished')

17:57:42: Writing processed file of 634679 rows to ../data/win_only_data-2021-09-14T17-57-42.csv
17:57:52: Finished
